In [1]:
import pandas as pd
import os

In [2]:
df = pd.read_parquet("D:/OneDrive - VNU-HCMUS/Introduction to AI/Tuần 3/silver_fhv_pickup.parquet")
df.head(20)

,pickup_datetime,PUlocationID,PickUpID
0,2023-01-01 00:18:35,206.0,202301774621
1,2023-01-01 00:36:19,NaN,202301115225
2,2023-01-01 00:19:18,NaN,202301340976
3,2023-01-01 00:08:14,NaN,202301239812
4,2023-01-01 01:28:36,NaN,202301699087
5,2023-01-01 01:51:16,NaN,202301830506
6,2023-01-01 01:32:44,NaN,202301759107
7,2023-01-01 01:10:35,42.0,202301404790
8,2023-01-01 01:35:16,41.0,202301160086
9,2023-01-01 02:52:30,NaN,202301208489


In [4]:
df = pd.read_parquet("D:/OneDrive - VNU-HCMUS/Introduction to AI/Tuần 3/silver_fhv_dropoff.parquet")
df.head(20)

,dropOff_datetime,DOlocationID,DropOffID
0,2023-01-01 00:31:59,39.0,202301618211
1,2023-01-01 01:39:04,119.0,202301663020
2,2023-01-01 01:25:57,252.0,202301841072
3,2023-01-01 01:59:44,71.0,202301978766
4,2023-01-01 02:23:07,89.0,202301375755
5,2023-01-01 01:31:25,239.0,202301886204
6,2023-01-01 02:07:01,147.0,202301702041
7,2023-01-01 01:41:38,148.0,202301136878
8,2023-01-01 01:43:29,179.0,202301909146
9,2023-01-01 01:47:02,74.0,202301339706
